In [5]:
import matplotlib.pyplot as plt
import pandas as pd
from src.SugarScape import SugarScape
from mesa import batch_run
import numpy as np
from tqdm import tqdm

# Setup all folders
import os

folder = 'Segregation'
# Create Segregation Folder
if not os.path.exists(folder):
    os.mkdir(folder)

# Check if folder exists for saving base results
folder = f'{folder}/Base'
if not os.path.exists(folder):
    os.mkdir(folder)

# Base Simulation

In [70]:
scenarios = [
    {'vision_mean': 4.264199728146195, 'metabolism_mean': 6.751366691663861, 'max_age_mean': 96.55008263885976,
     'repopulate_factor': 7.485186895355582, 'cell_regeneration': 1.6743798777461052},   # Worst case
    
    {'vision_mean': 2.571960593573749, 'metabolism_mean': 7.285224918276071, 'max_age_mean': 93.18476642481984,
     'repopulate_factor': 5.733334645628929, 'cell_regeneration': 2.95167014375329},     # Best case
    
    {'vision_mean': 1.2129357354715466, 'metabolism_mean': 3.5813456028699875, 'max_age_mean': 96.730439318344,
     'repopulate_factor': 12.009924734011292, 'cell_regeneration': 1.5217601098120213},  # Average case
    
    {'vision_mean': 3.5, 'metabolism_mean': 6, 'max_age_mean': 85,
     'repopulate_factor': 10, 'cell_regeneration': 3},                                   # Balanced case
    
    {'vision_mean': 1.2129357354715466, 'metabolism_mean': 3.5813456028699875, 'max_age_mean': 100,
     'repopulate_factor': 5, 'cell_regeneration': 1.5217601098120213},                    # Slow evolving
    
    {'vision_mean': 5.089661035686731, 'metabolism_mean': 5.488567331805825, 'max_age_mean': 70,
     'repopulate_factor': 15, 'cell_regeneration': 3.368525765836239},                    # Fast evolving
]

results = []
replicates = 30
max_steps = 10

In [71]:
for scenario in scenarios:
    scenario['map_scheme'] = 'split'
    scenario['track_scheme'] = 'segregation'
    
    # Run simulation
    batches = batch_run(SugarScape, scenario, number_processes=None,
                          iterations=replicates, max_steps=max_steps, display_progress=True, data_collection_period=1)
    
    for i in range(replicates):
        # Get sugar metabolisms
        scenario['lower_sugar'] = [float(batch['Lower Sugar Metabolism']) for batch in batches[(max_steps+1)*i:(max_steps+1)*(i+1)]]
        scenario['middle_sugar'] = [float(batch['Middle Sugar Metabolism']) for batch in batches[(max_steps+1)*i:(max_steps+1)*(i+1)]]
        scenario['upper_sugar'] = [float(batch['Upper Sugar Metabolism']) for batch in batches[(max_steps+1)*i:(max_steps+1)*(i+1)]]
        
        # Get spice metabolisms
        scenario['lower_spice'] = [float(batch['Lower Spice Metabolism']) for batch in batches[(max_steps+1)*i:(max_steps+1)*(i+1)]]
        scenario['middle_spice'] = [float(batch['Middle Spice Metabolism']) for batch in batches[(max_steps+1)*i:(max_steps+1)*(i+1)]]
        scenario['upper_spice'] = [float(batch['Upper Spice Metabolism']) for batch in batches[(max_steps+1)*i:(max_steps+1)*(i+1)]]
        
        scenario['lower_vision'] = [float(batch['Lower Vision']) for batch in batches[(max_steps+1)*i:(max_steps+1)*(i+1)]]
        scenario['middle_vision'] = [float(batch['Middle Vision']) for batch in batches[(max_steps+1)*i:(max_steps+1)*(i+1)]]
        scenario['upper_vision'] = [float(batch['Upper Vision']) for batch in batches[(max_steps+1)*i:(max_steps+1)*(i+1)]]
        
        results.append(scenario)

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

In [76]:
names = list(scenario.keys()) 
df = pd.DataFrame(results, columns=names)

# Save results
df.to_csv('Segregation/Base/results.csv')

# Experimenting with tax schemes

In [16]:
scenarios = [
    {'vision_mean': 4.264199728146195, 'metabolism_mean': 6.751366691663861, 'max_age_mean': 96.55008263885976,
     'repopulate_factor': 7.485186895355582, 'cell_regeneration': 1.6743798777461052},   # Worst case
    
    {'vision_mean': 2.571960593573749, 'metabolism_mean': 7.285224918276071, 'max_age_mean': 93.18476642481984,
     'repopulate_factor': 5.733334645628929, 'cell_regeneration': 2.95167014375329},     # Best case
    
    {'vision_mean': 1.2129357354715466, 'metabolism_mean': 3.5813456028699875, 'max_age_mean': 96.730439318344,
     'repopulate_factor': 12.009924734011292, 'cell_regeneration': 1.5217601098120213},  # Average case
    
    {'vision_mean': 3.5, 'metabolism_mean': 6, 'max_age_mean': 85,
     'repopulate_factor': 10, 'cell_regeneration': 3},                                   # Balanced case
    
    {'vision_mean': 1.2129357354715466, 'metabolism_mean': 3.5813456028699875, 'max_age_mean': 100,
     'repopulate_factor': 5, 'cell_regeneration': 1.5217601098120213},                    # Slow evolving
    
    {'vision_mean': 5.089661035686731, 'metabolism_mean': 5.488567331805825, 'max_age_mean': 70,
     'repopulate_factor': 15, 'cell_regeneration': 3.368525765836239},                    # Fast evolving
]

tax_systems = [
    ("progressive", "needs"),
    ("flat", "flat"),
    ("regressive", "random"),
    ("luxury", "progressive"),
    ("progressive", "progressive")
]

tax_rates = [0.1, 0.25, 0.4]

results = []
replicates = 3
max_steps = 2

In [17]:
with tqdm(total=len(scenarios), ncols=70) as pbar:
    for scenario in scenarios:
        for tax_system in tax_systems:
            for tax_rate in tax_rates:
                # Add parameters
                scenario_copy = scenario.copy()
                scenario_copy['tax_scheme'] = tax_system[0]
                scenario_copy['distributer_scheme'] = tax_system[1]
                scenario_copy['tax_rate'] = tax_rate
                scenario_copy['map_scheme'] = 'split'
                scenario_copy['track_scheme'] = 'segregation'
                
                # Run simulation
                batches = batch_run(SugarScape, scenario_copy, number_processes=None,
                                      iterations=replicates, max_steps=max_steps, display_progress=False, data_collection_period=1)
                
                for i in range(replicates):
                    # Get sugar metabolisms
                    scenario_copy['lower_sugar'] = [float(batch['Lower Sugar Metabolism']) for batch in batches[(max_steps+1)*i:(max_steps+1)*(i+1)]]
                    scenario_copy['middle_sugar'] = [float(batch['Middle Sugar Metabolism']) for batch in batches[(max_steps+1)*i:(max_steps+1)*(i+1)]]
                    scenario_copy['upper_sugar'] = [float(batch['Upper Sugar Metabolism']) for batch in batches[(max_steps+1)*i:(max_steps+1)*(i+1)]]
                    
                    # Get spice metabolisms
                    scenario_copy['lower_spice'] = [float(batch['Lower Spice Metabolism']) for batch in batches[(max_steps+1)*i:(max_steps+1)*(i+1)]]
                    scenario_copy['middle_spice'] = [float(batch['Middle Spice Metabolism']) for batch in batches[(max_steps+1)*i:(max_steps+1)*(i+1)]]
                    scenario_copy['upper_spice'] = [float(batch['Upper Spice Metabolism']) for batch in batches[(max_steps+1)*i:(max_steps+1)*(i+1)]]
                    
                    scenario_copy['lower_vision'] = [float(batch['Lower Vision']) for batch in batches[(max_steps+1)*i:(max_steps+1)*(i+1)]]
                    scenario_copy['middle_vision'] = [float(batch['Middle Vision']) for batch in batches[(max_steps+1)*i:(max_steps+1)*(i+1)]]
                    scenario_copy['upper_vision'] = [float(batch['Upper Vision']) for batch in batches[(max_steps+1)*i:(max_steps+1)*(i+1)]]
                    
                    results.append(scenario_copy)
        pbar.update(1)

100%|███████████████████████████████████| 6/6 [01:52<00:00, 18.74s/it]


In [18]:
names = list(scenario_copy.keys()) 
df = pd.DataFrame(results, columns=names)

# Save results
df.to_csv('Segregation/results.csv')

In [51]:
# fig, axs = plt.subplots(1, 3, figsize=(15, 5))
# keys = ['sugar', 'spice', 'vision']
# areas = ['lower', 'middle', 'upper']

# for ax, key in zip(axs, keys):
#     for area in areas:
#         # Get values
#         values = df[f'{area}_{key}']
        
#         # Convert to array
#         arr = np.vstack(values)
        
#         # Compute mean and ci
#         mean = np.mean(arr, axis=0)
#         ci = np.std(arr, ddof=1) / np.sqrt(arr.shape[0]) * 1.96
        
#         x = np.arange(len(mean))
#         ax.plot(x, mean, label=area.capitalize())
#         ax.fill_between(x, mean - ci, mean + ci, alpha=0.5)
    
#     ax.grid()
#     ax.set_title(key.capitalize())
    
# axs[-1].legend()
# fig.suptitle('Typical Scenario')
# fig.tight_layout()
    

30
